In [1]:
import os
from tqdm.auto import tqdm
from glob import glob
from PIL import Image, ImageDraw # draw on images
from IPython.display import display # display images on jupyter notebook
import numpy as np


In [2]:
input_dir = "RDD2022_0/Norway"
output_dir = "RDD2022_0/Norway1"
os.makedirs(output_dir, exist_ok=True)

In [3]:
input_train_images = os.path.join(input_dir, "train/images")
input_test_images = os.path.join(input_dir, "test/images")
input_train_annotations = os.path.join(input_dir, "train/annotations/xmls")

# Check Image Sizes

In [4]:
files = glob(os.path.join(input_train_images, "*.jpg"))
widths = set()
heights = set() 
for f in tqdm(files):
    img = Image.open(f)
    w, h = img.size
    widths.add(w)
    heights.add(h)


In [5]:
widths

{3643, 3650, 4040}

In [6]:
heights

{2035, 2041, 2044}

In [7]:
output_train_images = os.path.join(output_dir, "train/images")
output_test_images = os.path.join(output_dir, "test/images")
output_train_annotations = os.path.join(output_dir, "train/annotations/xmls")
os.makedirs(output_train_images, exist_ok=True)
os.makedirs(output_test_images, exist_ok=True)
os.makedirs(output_train_annotations, exist_ok=True)

# Crop the images

In [8]:
img_size = 1824
# cut img
def cut_img(file):
    img = Image.open(file)
    img_w, img_h = img.size
    img = img.crop((0, img_h-img_size, img_size, img_h))
    # save it
    img.save(file.replace(input_dir, output_dir))

In [9]:
files = glob(os.path.join(input_train_images, "*.jpg"))
for file in tqdm(files):
    cut_img(file)

In [10]:
files = glob(os.path.join(input_test_images, "*.jpg"))
for file in tqdm(files):
    cut_img(file)

# Crop the annotations


In [11]:
import xml.etree.ElementTree as ET # parse xml anotation


In [12]:
def cut_anno(file, new_width=1824, new_height=1824):
    out_range = 0
    in_range = 0
    # parse the content of the xml file
    tree = ET.parse(file)
    root = tree.getroot()
    width = int(root.find("size").find("width").text)
    height = int(root.find("size").find("height").text)
    root.find("size").find("width").text = str(new_width)
    root.find("size").find("height").text = str(new_height)
    
    for obj in root.findall('object'):
#         import pdb
#         pdb.set_trace()
        xmin = obj.find("bndbox").find("xmin")
        ymin = obj.find("bndbox").find("ymin")
        xmax = obj.find("bndbox").find("xmax")
        ymax = obj.find("bndbox").find("ymax")
        # convert to ints
        ymin_val = int(float(ymin.text)) - (height-new_height)
        ymax_val = int(float(ymax.text)) - (height-new_height)
        xmax_val = int(float(xmax.text))
        xmin_val = int(float(xmin.text))
        # crop the boxes
        if ymin_val < 0:
            ymin_val = 0
            
        if xmax_val > new_width:
            xmax_val = new_width
        
        h = ymax_val - ymin_val
        w = xmax_val - xmin_val
        
        # if the box is out of range remove it
        if ymax_val <= 0 or xmin_val >= new_width or h<=0 or w <= 0 or h*w <= 20: # too small
            root.remove(obj)
            out_range += 1
        else:
            xmin.text = str(xmin_val)
            ymin.text = str(ymin_val)
            xmax.text = str(xmax_val)
            ymax.text = str(ymax_val)
            in_range += 1
    # write output
    tree.write(file.replace(input_dir, output_dir))
    return out_range, in_range

In [13]:
files = glob(os.path.join(input_train_annotations, "*.xml"))

In [14]:
len(files)

8161

In [15]:
total_out = 0
total_in = 0
for file in tqdm(files):
    o, i = cut_anno(file)
    total_out += o
    total_in += i

In [16]:
total_out, total_in

(537, 10692)